In [ ]:
!pip show roboflow ultralytics


In [1]:
from roboflow import Roboflow
import os
from ultralytics import YOLO
from datetime import datetime
import cv2
from pathlib import Path
import shutil
from glob import glob

## Descarregar el dataset

In [ ]:
rf = Roboflow(api_key="5HYzrdPm6LO8xmXDVn3G")
project = rf.workspace("vc").project("matriculas-espanolas")
version = project.version(1)
dataset = version.download("yolov11")                

## Entrenar el model

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(data=os.path.join(dataset.location, "data.yaml"), epochs=50, imgsz=640)

## Guardar el model

In [ ]:
models_path = "models"
fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
model_name = f"model_{fecha_hora_actual}"
model_path = os.path.join(models_path, model_name)
os.makedirs(models_path, exist_ok=True)
model.save(f"{model_path}.pt")

## Carregar el model

In [2]:
# Configuración
model_path = "./models/model_27_03_2025_19_30.pt"
input_path = "./Matriculas-Españolas-1/valid/images"
output_dir = "./predicted"
os.makedirs(output_dir, exist_ok=True)

# Cargar modelo
model_matr = YOLO(model_path)

## Prediure imagues

In [7]:
import os
from glob import glob  # Importamos la función glob directamente

def get_image_paths(input_path):
    """Obtiene la lista de rutas de imágenes a procesar"""
    if isinstance(input_path, list):
        return input_path
    elif os.path.isfile(input_path):
        return [input_path]
    elif os.path.isdir(input_path):
        image_paths = (glob(os.path.join(input_path, '*.jpg')) + 
                      glob(os.path.join(input_path, '*.png')) + 
                      glob(os.path.join(input_path, '*.jpeg')))
        return image_paths
    else:
        raise ValueError("Input path no es válido (debe ser imagen, lista o carpeta)")

def generar_markdown_imagenes(input_dir, output_md_path):
    """Genera un archivo Markdown con todas las imágenes de un directorio"""
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"El directorio {input_dir} no existe")
    
    imagenes = sorted(glob(os.path.join(input_dir, '*.jpg')) + 
                    glob(os.path.join(input_dir, '*.png')) + 
                    glob(os.path.join(input_dir, '*.jpeg')))
    
    contenido_md = "# Imágenes generadas\n\n"
    for img_path in imagenes:
        img_name = os.path.basename(img_path)
        ruta_relativa = os.path.relpath(img_path, start=os.path.dirname(output_md_path))
        contenido_md += f"## {img_name}\n\n"
        contenido_md += f"![{img_name}]({ruta_relativa})\n\n"
    
    os.makedirs(os.path.dirname(output_md_path), exist_ok=True)
    
    with open(output_md_path, 'w') as md_file:
        md_file.write(contenido_md)
    
    return output_md_path

In [49]:
input_path = "Matriculas-Españolas-1/valid/images/"
image_paths = get_image_paths(input_path)

name = "images"
project = "./output_model"
results = model_matr.predict(
    source=image_paths,
    conf=0.25,
    save=True,
    save_txt=True,
    name = name,
    project = project,
    exist_ok=True,
    verbose=False
)

origen = os.path.join(project, name, "labels")
destino = os.path.join(project, "labels")

shutil.copytree(origen, destino, dirs_exist_ok=True)
shutil.rmtree(origen)

md_path = os.path.join(project, "imagenes_generadas.md")
image_paths = os.path.join(project, name)

generar_markdown_imagenes(image_paths, md_path)

Results saved to output_model/images
102 labels saved to output_model/images/labels


'./output_model/imagenes_generadas.md'

## Retallar matricules

In [12]:
def predecir_yolo(photo, model):
    results = model.predict(
        source=photo,
        conf=0.25,
        save=False,
        verbose=False
    )
    return results[0]

def obtener_bbox(result):
    recuadros = []
    for box in result.boxes:
        recuadros.append(box.xyxy)
    return recuadros

def recortar_y_guardar_matricula(bboxes, photo, output_dir="recortes_matriculas"):
    """Recorta las matriculas y las guarda en una carpeta"""
    img = cv2.imread(photo)
    if img is None:
        raise ValueError(f"No se pudo leer la imagen: {photo}")
    
    recortes = []
    for i, bbox in enumerate(bboxes):
        coords = bbox.tolist()[0]
        x1, y1, x2, y2 = map(int, coords)
        
        # Validar coordenadas
        h, w = img.shape[:2]
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        if x2 > x1 and y2 > y1:
            recorte = img[y1:y2, x1:x2]
            recortes.append(recorte)
            
            # Guardar inmediatamente cada recorte
            os.makedirs(output_dir, exist_ok=True)
            ruta_guardado = os.path.join(output_dir, f"{os.path.basename(photo)}.jpg")
            cv2.imwrite(ruta_guardado, recorte)
            print(f"Guardado: {ruta_guardado}")
        else:
            print(f"Advertencia: Bbox inválida {[x1, y1, x2, y2]}")
    
    return recortes

In [ ]:
import os
import cv2

img = "./Matriculas-Españolas-1/valid/images/1668448029089_jpg.rf.779ea16b2d2e47a624e8ebfa0d3e96a9.jpg"
img = "./Matriculas-Españolas-1/valid/images/"
output_folder = "./matriculas_recortadas"

imges_path = get_image_paths(img)
for img_path in imges_path:    
    result = predecir_yolo(img_path, model_matr)
    recuadros = obtener_bbox(result)
    img_recortadas = recortar_y_guardar_matricula(recuadros, img_path, output_folder)

Guardado: ./matriculas_recortadas/1668448029089_jpg.rf.779ea16b2d2e47a624e8ebfa0d3e96a9.jpg.jpg
Guardado: ./matriculas_recortadas/1668451259688_jpg.rf.dd34dbc217c4a1baff18f22c1c47bc55.jpg.jpg
Guardado: ./matriculas_recortadas/1668451259954_jpg.rf.f9661736cda912fc4a5d9f5970a692e7.jpg.jpg
Guardado: ./matriculas_recortadas/1668451259741_jpg.rf.592f6d6c4dacade63ec22118f0ceea75.jpg.jpg
Guardado: ./matriculas_recortadas/1668451259641_jpg.rf.c0283e246e83147b150fa0a0557a4111.jpg.jpg
Guardado: ./matriculas_recortadas/1668451259861_jpg.rf.635cbf891981ca61bd2cdd67f42b2da2.jpg.jpg

Imágenes guardadas
